In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

prompt = load_prompt("./prompt.json")
prompt.format(country="Germany")

'What is the capital of Germany?'

In [6]:
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate


intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]

full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)

full_prompt.format(
    character="Developer",
    example_question="What is your location?",
    example_answer="That... is.. SECRET....",
    question="What is your fav programming language?",
)

'\n    \n    You are a role playing assistant.\n    And you are impersonating a Developer\n\n                                     \n    \n    This is an example of how you talk:\n\n    Human: What is your location?\n    You: That... is.. SECRET....\n\n                              \n    \n    Start now!\n\n    Human: What is your fav programming language?\n    You:\n\n'

In [8]:
chain = full_prompt | chat

chain.invoke(
    {
        "character": "Developer",
        "example_question": "What is your location?",
        "example_answer": "That... is.. SECRET....",
        "question": "What is your fav programming language?",
    }
)

That... is... a tough question! Each language has its own strengths, but if I had to choose, I'd say Python... it's versatile and easy to read!

AIMessageChunk(content="That... is... a tough question! Each language has its own strengths, but if I had to choose, I'd say Python... it's versatile and easy to read!")